In [1]:
import  numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


randomseed=42
np.random.seed(randomseed)

In [2]:

xtest=np.array(pd.read_csv('../dataset/xtest.txt'))
xtrain=np.array(pd.read_csv('../dataset/xtrain.txt'))
ytest_original=np.array(pd.read_csv('../dataset/ytest.txt')).ravel()
ytrain_original =np.array(pd.read_csv('../dataset/ytrain.txt')).ravel()

# data=datasets.load_wine()
# x=data.data
# y=data.target

# data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))

# data = pd.read_csv("../dataset/ionosphere.data",  header=None)
# data = shuffle(data)
# le = LabelEncoder()
# data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
# x = data.iloc[:, :-1]
# y = data.iloc[:, -1]
# print(np.unique(y))




# xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 

ytrain=ytrain_original.copy()
ytest=ytest_original.copy() 

# ytrain, ytest = swapcolumns(ytrain, ytest, 2)

In [3]:


clf=[]
acc=[]
finalacc=[]
ypredproba_all=[]
ypredconfprob_all=[]


#================================================= 

# Class 0
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,0)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))


xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)
xgbpred=xgbc.predict(xtest)
print('original score 2',m.f1_score(ytest,xgbpred,average='weighted'))


#================================================= 

rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


#=================================================
svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
svc.fit(xtrain,ytrain)

svcpred=svc.predict(xtest)
print(m.f1_score(ytest,svcpred,average='weighted'))

clf.append(svc)
acc.append(m.f1_score(ytest,svcpred,average='weighted'))
ypredproba_all.append(svc.predict_proba(xtest)[:,1])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)

#=================================================
xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
xgbc.fit(xtrain,ytrain)

xgbpred=xgbc.predict(xtest)
print(m.f1_score(ytest,xgbpred,average='weighted'))


clf.append(xgbc)
acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
ypredproba_all.append(xgbc.predict_proba(xtest)[:,0])

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat.ravel() / 100)


original score 0.7989010989010988
original score 2 0.8173286512555166
0.7989010989010988
0.7762805187278128
0.8173286512555166


In [4]:

#=================================================
#=================================================
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(50):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)
    
   


[184, 52, 256, 318, 281, 485, 141, 719, 361, 401, 708, 775, 280, 689, 33, 615, 213, 691, 43, 101, 318, 147, 296, 149, 385, 741, 493, 713, 90, 296, 344, 546, 343, 56, 281, 633, 709, 352, 243, 576, 644, 334, 92, 248, 785, 239, 120, 131, 236, 400]
0  ====================  [ 0  1  3  6  7  9 10]
0.8009506833036245
1  ====================  [ 0  1  2  3  8  9 10]
0.7974008677561183
2  ====================  [ 0  2  3  4  5  6 11]
0.7916804956300475
3  ====================  [ 0  2  3  7  8  9 11]
0.7960892559633282
4  ====================  [ 0  2  3  4  7  9 11]
0.8013897787196294
5  ====================  [ 1  2  3  4  6  9 11]
0.7540327890943949
6  ====================  [0 1 3 4 6 7 8]
0.8082137896276629
7  ====================  [ 2  3  5  7  8 10 11]
0.7222406608400633
8  ====================  [ 0  2  5  6  7  9 11]
0.7858999662278351
9  ====================  [ 0  3  4  6  7  9 10]
0.7879180817124627
10  ====================  [ 2  3  5  6  7  8 10]
0.6992468359344669
11  ====================

In [5]:

for i in range(len(comb)):
    print(i, " ==================== ", comb[i])
    
    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain[:, comb[i]], ytrain)
    xgbmodelpred = xgbmodel.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest[:, comb[i]])[:,0])

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat.ravel() / 100)
    
    #"====================================================="



0  ====================  [ 0  1  3  6  7  9 10]
0.818133766026232
1  ====================  [ 0  1  2  3  8  9 10]
0.8124550091763205
2  ====================  [ 0  2  3  4  5  6 11]
0.8120132050192357
3  ====================  [ 0  2  3  7  8  9 11]
0.8144283214984048
4  ====================  [ 0  2  3  4  7  9 11]
0.8086860051080718
5  ====================  [ 1  2  3  4  6  9 11]
0.785133536621525
6  ====================  [0 1 3 4 6 7 8]
0.8024968680706386
7  ====================  [ 2  3  5  7  8 10 11]
0.7700083949770786
8  ====================  [ 0  2  5  6  7  9 11]
0.8159829870888571
9  ====================  [ 0  3  4  6  7  9 10]
0.8020316052857438
10  ====================  [ 2  3  5  6  7  8 10]
0.7247107772474757
11  ====================  [ 3  4  6  8  9 10 11]
0.768240529686313
12  ====================  [ 0  2  3  4  7  9 10]
0.8053588051969078
13  ====================  [ 2  3  4  5  8  9 11]
0.7659238082153843
14  ====================  [ 0  1  2  3  5  9 10]
0.8140073540056169


In [6]:

# for i in range(len(comb)):
#     print(i, " ==================== ", comb[i])

#     svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
#     svc.fit(xtrain[:, comb[i]], ytrain)
#     svcpred=svc.predict(xtest[:, comb[i]])
    
#     print(m.f1_score(ytest,svcpred,average='weighted'))

#     clf.append(svc)
    
#     acc.append(m.f1_score(ytest,svcpred,average='weighted'))
#     ypredproba_all.append(svc.predict_proba(xtest[:, comb[i]])[:,1])

#     confmat = m.confusion_matrix(ytest, svcpred)
#     confsumh = np.sum(confmat, axis=1)
#     propconfmat = confmat.copy()
#     for i in range(propconfmat.shape[0]):
#         propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
#     ypredconfprob_all.append(propconfmat.ravel() / 100)

# # #=================================================


In [7]:
# np.savetxt("foo.csv", np.array(ypredproba_all), delimiter=",")

In [8]:
# pd_clf=pd.DataFrame(list(clf))
pd_acc=pd.DataFrame(acc) 
pd_ypredproba_all=pd.DataFrame(np.array(ypredproba_all))
pd_ypredconfprob_all=pd.DataFrame(ypredconfprob_all)

In [9]:
classname='class0.txt'
pd_acc.to_csv('pd_acc_'+classname,sep=',',index=False) 
pd_ypredproba_all.to_csv('pd_ypredproba_all_'+classname,sep=',',index=False)
pd_ypredconfprob_all.to_csv('pd_ypredconfprob_all_'+classname,sep=',',index=False)

In [10]:
pd.DataFrame(ytest_original).to_csv('ytest_original.txt',sep=',',index=False)
pd.DataFrame(ytest).to_csv('ytest_class0.txt',sep=',',index=False)